<a href="https://colab.research.google.com/github/cfoli/Multi-label-Medical-Image-Classification/blob/main/ChestVision_ViTs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Vision-Transformer-based solutions for assistive medical diagnosis

This project leverages pre-trained vision transformers to build end-to-end solutions for multi-label medical image (i.e., chest x-ray) classification.

### Import libraries

---



In [1]:
!pip install lightning torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.6/853.6 kB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.3/857.3 kB 68.5 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn.functional as F
import torchvision
from torchvision import transforms, models, datasets
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from torch.utils.data import random_split
import pytorch_lightning as torch_light
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
import torchmetrics
from torchmetrics import Metric
import os
import shutil
import subprocess
import pandas as pd
from PIL import Image
from transformers import AutoModel


### Define helper functions and containers

---



In [3]:
BASE_DIR = os.path.join(os.getcwd(), "datasets")

configs = {
    "IMAGE_SIZE":   (224, 224),    # Resize images to (W, H)
    "NUM_CHANNELS": 3,             # RGB images
    "NUM_CLASSES":  15,            # Number of output labels

    "BATCH_SIZE": 64,
    "NUM_WORKERS": 10,

    # ImageNet dataset normalization values (for pretrained backbones)
    "MEAN": (0.485, 0.456, 0.406),
    "STD":  (0.229, 0.224, 0.225),

    "LABELS_CSV_DIR": os.path.join(BASE_DIR, "new_labels.csv"),
    "IMG_DIR": os.path.join(BASE_DIR, "resized_images", "resized_images"),

    "TRAIN_PCT": 0.8, # Fraction of data for training
    "VAL_PCT":   0.1, # Fraction of data for validation
    }

# labels = ["Atelectasis", "Cardiomegaly", "Consolidation", "Edema", "Effusion", "Emphysema", "Fibrosis", "Hernia", "Infiltration", "Mass", "No finding", "Nodule", "Pleural_Thickening", "Pneumonia","Pneumothorax"]

MODEL_REGISTRY = {
    "CheXFormer-base": "m42-health/CXformer-base",
    "CheXFormer-small": "m42-health/CXformer-small",
    "ViT-base-16": "google/vit-base-patch16-224",
    "ViT-large-16": "google/vit-large-patch16-224",

}


In [4]:
model = torchvision.models.efficientnet_b3(weights="DEFAULT")
# print(model) # efficientnet_v2_s, AlexNet
last_module_name, last_module = list(model.named_children())[-1]
print(last_module)


Downloading: "https://download.pytorch.org/models/efficientnet_b3_rwightman-b3899882.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b3_rwightman-b3899882.pth


100%|██████████| 47.2M/47.2M [00:00<00:00, 224MB/s]


Sequential(
  (0): Dropout(p=0.3, inplace=True)
  (1): Linear(in_features=1536, out_features=1000, bias=True)
)


In [17]:
from transformers import AutoModel, AutoImageProcessor
from PIL import Image

model_name = "m42-health/CXformer-base"
# Options (Multi-label image classification):
# m42-health/CXformer-base,
# m42-health/CXformer-small
# google/vit-base-patch16-224
# facebook/dinov3-vitl16-pretrain-lvd1689m
# facebook/dinov3-vitb16-pretrain-lvd1689m

image_processor = AutoImageProcessor.from_pretrained(model_name,trust_remote_code=True)
model = AutoModel.from_pretrained(model_name)

# Options (Image-to-text)
# nlpconnect/vit-gpt2-image-captioning


Loading weights:   0%|          | 0/224 [00:00<?, ?it/s]

In [18]:
# print(model)
# last_module_name, last_module = list(model.named_children())[-1]
model.config.hidden_size

768

In [24]:
class get_pretrained_model(nn.Module):
    def __init__(
        self,
        model_name: str,
        num_classes: int,
        num_layers_to_unfreeze: int = 0):
        super().__init__()

        print(f"Loading pretrained [{model_name}] model")

        self.backbone = AutoModel.from_pretrained(
            MODEL_REGISTRY[model_name],
            trust_remote_code=True)

        hidden_size = self.backbone.config.hidden_size

        # Freeze entire backbone first
        for param in self.backbone.parameters():
            param.requires_grad = False

        # Selectively unfreeze last N layers
        if num_layers_to_unfreeze > 0:
            self._unfreeze_last_n_layers(num_layers_to_unfreeze)

        # Single classification head
        self.classifier = nn.Sequential(
            nn.LayerNorm(hidden_size),
            nn.Dropout(0.2),
            nn.Linear(hidden_size, num_classes) )

    def forward(self, x):
        outputs = self.backbone(x)

        # Use CLS token
        img_embeddings = outputs.last_hidden_state[:, 0]

        logits = self.classifier(img_embeddings)
        return logits

    def _unfreeze_last_n_layers(self, n: int):
        if hasattr(self.backbone, "encoder"):
            encoder_layers = self.backbone.encoder.layer
        elif hasattr(self.backbone, "vision_model"):
            encoder_layers = self.backbone.vision_model.encoder.layer
        else:
            raise ValueError("Cannot find encoder layers in backbone.")

        total_layers = len(encoder_layers)
        n = min(n, total_layers)

        print(f"Unfreezing last {n} of {total_layers} transformer layers.")

        for layer in encoder_layers[-n:]:
            for param in layer.parameters():
                param.requires_grad = True


In [ ]:

model.eval()

image = Image.open('sample_cxr.png')

image = image_processor(image, return_tensors='pt')
print(image['pixel_values'].shape) # [1,3,518,518]

print("Doing forwardpass...")
output = model(**image).last_hidden_state  # [1, 1374, 768]


### Load kaggle credentials (i.e., API) file

---



In [5]:
# Import kaggle API token file
from google.colab import files
files.upload() # This will prompt you to select and upload the file

# Then move and secure the file
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle.json


### Create model (i.e. classifier) module

---



In [25]:
class modelModule(torch_light.LightningModule):
    def __init__(self, num_classes, backbone_model_name, num_layers_to_unfreeze):
        super().__init__()
        self.num_classes = num_classes
        self.backbone_model_name = backbone_model_name
        self.num_layers_to_unfreeze = num_layers_to_unfreeze

        # Load a pretrained backbone and replace its final layer
        self.model = get_pretrained_model(
            num_classes = self.num_classes,
            model_name  = self.backbone_model_name,
            num_layers_to_unfreeze = self.num_layers_to_unfreeze)

        # Binary classification loss operating on raw logits
        self.loss_function      = torch.nn.BCEWithLogitsLoss()
        # self.accuracy_function = torchmetrics.Accuracy(task="multilabel", num_labels=self.num_classes)
        # self.f1_score_function = torchmetrics.F1Score(task="multilabel", num_labels=self.num_classes)
        self.accuracy_function  = torchmetrics.classification.MultilabelAccuracy(num_labels=self.num_classes, average="weighted", threshold=0.5)
        self.f1_score_function  = torchmetrics.classification.MultilabelF1Score(num_labels=self.num_classes, average="weighted", threshold=0.5)
        self.auroc_function     = torchmetrics.classification.MultilabelAUROC(num_labels=self.num_classes, average="weighted", thresholds=10)
        self.map_score_function = torchmetrics.classification.MultilabelAveragePrecision(num_labels=self.num_classes, average="weighted", thresholds=10)
        # average options: macro (simple average), micro (sum), weighted (weight by class size, then avg)
        # threshold: Threshold for transforming probability to binary (0,1) predictions. For some metrics (e.g., AUROC), represents the number of thresholds (evenly spaced b/n 0–1) the metric should be computed at (resulting array of values are the averaged to obtain the final score)

    def forward(self, x):
        # Forward pass through the backbone model
        return self.model(x)

    def _common_step(self, batch, batch_idx):
        """
        Shared logic for train / val / test steps.
        Computes loss and evaluation metrics.
        """
        x, y = batch

        # Compute model predictions ()
        y_logits = self.forward(x)
        y_prob    = torch.sigmoid(y_logits)

        # Compute metrics (expects logits + labels)
        loss     = self.loss_function(y_logits, y.float())

        # Compute mean loss over all classes
        # loss     = torchmetrics.aggregation.MeanMetric(self.loss_function(y_hat, y.float()), weight=X.shape[0])
        accuracy = self.accuracy_function(y_prob, y)
        f1_score = self.f1_score_function(y_prob, y)
        auroc    = self.auroc_function(y_prob, y)
        mAP      = self.map_score_function(y_prob, y) # mean average precision

        return loss, y_logits, y, accuracy, f1_score, auroc, mAP

    def training_step(self, batch, batch_idx):
        # Run shared step
        loss, y_logits, y, accuracy, f1_score, auroc, mAP = self._common_step(batch, batch_idx)

        # Log epoch-level training metrics
        self.log_dict(
            {"train_loss": loss, "train_accuracy": accuracy, "train_f1_score": f1_score, "train_auroc": auroc, "train_mAP": mAP},
            on_step=False, on_epoch=True, prog_bar=True)

        # Lightning expects the loss key for backprop
        return {"loss": loss}

    def validation_step(self, batch, batch_idx):
        # Run shared step
        loss, y_logits, y, accuracy, f1_score, auroc, mAP = self._common_step(batch, batch_idx)

        # Log validation metrics
        self.log_dict(
            {"val_loss": loss, "val_accuracy": accuracy,"val_f1_score": f1_score, "val_auroc": auroc, "val_mAP": mAP},
            on_step=False, on_epoch=True, prog_bar=True)

    def test_step(self, batch, batch_idx):
        # Run shared step
        loss, y_logits, y, accuracy, f1_score, auroc, mAP = self._common_step(batch, batch_idx)

        # Log test metrics
        self.log_dict(
            {"test_loss": loss, "test_accuracy": accuracy,"test_f1_score": f1_score, "test_auroc": auroc, "test_mAP": mAP},
            on_step=False, on_epoch=True, prog_bar=True)

    def predict_step(self, batch, batch_idx):
        """
        Prediction logic used by trainer.predict().
        Returns model outputs without computing loss.
        """
        x = batch if not isinstance(batch, (tuple, list)) else batch[0]
        logits = self.forward(x)

        # Convert logits to probabilities for inference
        probs = torch.sigmoid(logits)

        return probs

    def configure_optimizers(self):
        # Optimizer over all trainable parameters
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer


### Create data module

---



In [7]:

class createMultiLabelTorchDataset(Dataset):
    def __init__(self, labels_csv_dir, img_dir, transform=None):
        self.df = pd.read_csv(labels_csv_dir)
        self.label_cols = self.df.columns.drop("Path") # Get names of labels (i.e., cols, excluding img name)
        self.img_dir     = img_dir
        self.transform   = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_id = row["Path"]

        img_path = os.path.join(self.img_dir, image_id)
        image = Image.open(img_path).convert("RGB")  # Forces RGB (3 channels)

        if self.transform:
            image = self.transform(image)

        # multi-hot target
        target = row[self.label_cols].values.astype(int) # extract multihot target array from csv file and convert to int
        target = torch.tensor(target, dtype=torch.long) # convert multihot target array to long tensor

        return image, target

class dataModule(torch_light.LightningDataModule):
  def __init__(self, batch_size, num_classes):
    super().__init__()

    self.num_classes = num_classes
    # self.validation_size = validation_size # validation_size = 0.1
    self.batch_size = batch_size

    self.train_transforms = transforms.Compose([
                transforms.RandomAffine(degrees=40, translate=(0.01, 0.12), shear=0.05),
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.Normalize(configs["MEAN"], configs["STD"], inplace=True),
                transforms.ToTensor(),
                transforms.RandomErasing(inplace=True)])

    self.valid_transforms = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(configs["MEAN"], configs["STD"], inplace=True),
                ])
    self.test_transforms = self.valid_transforms

  def prepare_data(self):
      # Download datasets from Kaggle if not already downloaded
      if os.path.exists(BASE_DIR):
          return  # already downloaded

      # Kaggle expects the API key (i.e., json) to live in a directory like ~/.kaggle/kaggle.json. So we create that directory and copy the loaded Kaggle file there
      kaggle_dir  = os.path.join(os.path.expanduser("~"), ".kaggle")
      kaggle_json = os.path.join(kaggle_dir, "kaggle.json")

      if not os.path.exists(kaggle_json):
          os.makedirs(kaggle_dir, exist_ok=True)
          shutil.copyfile("kaggle.json", kaggle_json)
          os.chmod(kaggle_json, 0o600)

      print("Downloading dataset from Kaggle...")

      # Download datasets from Kaggle
      subprocess.run(
          ["kaggle", "datasets", "download","-q", "rahulogoel/nih-balanced-and-resized-chest-x-rays", "-p", BASE_DIR, "--unzip"],
          check=True)
      print("Download complete!")

  def setup(self, stage=None):

    full_dataset = createMultiLabelTorchDataset(
        labels_csv_dir = configs["LABELS_CSV_DIR"],
        img_dir = configs["IMG_DIR"],
        transform = self.train_transforms # temporarily assign train transforms
        )

    n_total = len(full_dataset)
    train_set_size = int(configs["TRAIN_PCT"] * n_total)
    val_set_size   = int(configs["VAL_PCT"] * n_total)
    test_set_size  = n_total - train_set_size - val_set_size

    generator = torch.Generator().manual_seed(42)

    train_subset, val_subset, test_subset = random_split(
        full_dataset,[train_set_size, val_set_size, test_set_size],
        generator=generator)

    # Assign transforms AFTER split
    train_subset.dataset.transform = self.train_transforms
    val_subset.dataset.transform   = self.valid_transforms
    test_subset.dataset.transform  = self.test_transforms

    if stage in (None, "fit"):
        self.train_dataset = train_subset
        self.val_dataset   = val_subset

    if stage in (None, "test"):
        self.test_dataset  = test_subset


  def train_dataloader(self):
    # create training data loader object from training dataset
    return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers = configs["NUM_WORKERS"])

  def val_dataloader(self):
    return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False, num_workers = configs["NUM_WORKERS"])

  def test_dataloader(self):
    return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False, num_workers = configs["NUM_WORKERS"])

### Train model

---



In [27]:
model_ = modelModule(
    num_classes = configs["NUM_CLASSES"],
    backbone_model_name = "CheXFormer-base",
    num_layers_to_unfreeze = 0 )
    # efficientnet_v2_s, convnext_small, alexnet, resnet50, vgg11

data_module = dataModule(
    batch_size      = configs["BATCH_SIZE"],
    num_classes     = configs["NUM_CLASSES"])

trainer_ = torch_light.Trainer(
    accelerator = "auto",
    devices     = "auto",
    strategy    = "auto",
    max_epochs  = 5,
    callbacks   = [EarlyStopping(patience=2, verbose=False, monitor="val_loss")],
    precision   = "16")

trainer_.fit(model_, data_module)
trainer_.validate(model_, data_module)
trainer_.test(model_, data_module)


Loading pretrained [CheXFormer-base] model


Loading weights:   0%|          | 0/224 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/lightning_fabric/connector.py:571: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.

┏━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name               ┃ Type                       ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ model              │ get_pretrained_model       │ 86.6 M │ train │     0 │
│ 1 │ loss_function      │ BCEWithLogitsLoss          │      0 │ train │     0 │
│ 2 │ accuracy_function  │ MultilabelAccuracy         │      0 │ train │     0 │
│ 3 │ f1_score_function  │ MultilabelF1Score          │      0 │ train │     0 │
│ 4 │ auroc_function     │ MultilabelAUROC            │      0 │ train │     0 │
│ 5 │ map_score_function │ MultilabelAveragePrecision │      0 │ train │     0 │
└───┴────────────────────┴────────────────────────────┴────────┴───────┴───────┘

Trainable params: 13.1 K                                                                                           
Non-trainable params: 86.6 M                                                                                       
Total params: 86.6 M                                                                                               
Total estimated model params size (MB): 346                                                                        
Modules in train mode: 10                                                                                          
Modules in eval mode: 224                                                                                          
Total FLOPs: 0

...

/usr/local/lib/python3.12/dist-packages/pytorch_lightning/loops/fit_loop.py:534: Found 224 module(s) in eval mode 
at the start of training. This may lead to unexpected behavior during training. If this is intentional, you can 
ignore this warning.

INFO:pytorch_lightning.utilities.rank_zero:
Detected KeyboardInterrupt, attempting graceful shutdown ...


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/call.py", line 49, in _call_and_handle_interrupt
    return trainer_fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/trainer.py", line 630, in _fit_impl
    self._run(model, ckpt_path=ckpt_path, weights_only=weights_only)
  File "/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/trainer.py", line 1079, in _run
    results = self._run_stage()
              ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/trainer.py", line 1123, in _run_stage
    self.fit_loop.run()
  File "/usr/local/lib/python3.12/dist-packages/pytorch_lightning/loops/fit_loop.py", line 217, in run
    self.advance()
  File "/usr/local/lib/python3.12/dist-packages/pytorch_lightning/loops/fit_loop.py", line 465, in advance
    self.epoch_loop.run(self._data_fetcher)
  File

TypeError: object of type 'NoneType' has no len()

In [12]:
model = getattr(torchvision.models, "alexnet")(weights="DEFAULT")

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


100%|██████████| 233M/233M [00:01<00:00, 238MB/s]


In [23]:
list(model.parameters())[-4].shape

torch.Size([4096, 4096])

In [ ]:
if freeze_all_layers:
    for param in self.backbone.parameters():
        param.requires_grad = False
else:
    n_layers = len(list(model.parameters()))
    for ix, param in enumerate(self.backbone.parameters()):
        if ix < n_layers - unfreeze_last_N:
            param.requires_grad = True
        else:
            param.requires_grad = False


### Train and compare different models

In [ ]:

SAVE_ROOT = '/content/drive/MyDrive/ML Projects/Multi-label Diagnosis (ChestXRay)'

os.makedirs(SAVE_ROOT, exist_ok=True)

# Selection criteria: top1 acc>85%; GFLOPS<500:
backbones = ["ViT_L_16", "ViT_B_16", "efficientnet_b3",  "efficientnet_v2_s", "regnet_x_3_2gf", "resnet50"]
# best overall: regnet_y_32gf
results = []

for backbone in backbones:
    print(f"\n================ {backbone} =================\n")

    save_dir = os.path.join(SAVE_ROOT, "Trained models")
    os.makedirs(save_dir, exist_ok=True)

    # ------------------
    # Model
    # ------------------
    model_ = modelModule(
        num_classes         = configs["NUM_CLASSES"],
        backbone_model_name = backbone)

    data_module = dataModule(
        batch_size  = configs["BATCH_SIZE"],
        num_classes = configs["NUM_CLASSES"])

    # ------------------
    # Callbacks
    # ------------------
    checkpoint_cb = ModelCheckpoint(
        dirpath    = save_dir,
        filename   = f"{backbone}",
        monitor    = "val_loss",        # 🔥 use mAP as primary metric
        mode       = "max",
        save_top_k = 1,
        save_last  = False)

    early_stop_cb = EarlyStopping(
        monitor   = "val_loss",
        mode      = "max",
        patience  = 5)

    # ------------------
    # Trainer
    # ------------------
    trainer_ = torch_light.Trainer(
        accelerator         = "auto",
        devices             = "auto",
        strategy            = "auto",
        max_epochs          = 15,
        precision           = "16",
        callbacks           = [checkpoint_cb, early_stop_cb],
        enable_progress_bar = True,
        logger              = False)

    # ------------------
    # Train
    # ------------------
    trainer_.fit(model_, data_module)

    # ------------------
    # Load best model
    # ------------------
    best_ckpt_path = checkpoint_cb.best_model_path
    print(f"Best checkpoint: {best_ckpt_path}")

    best_model = modelModule.load_from_checkpoint(
        best_ckpt_path,
        num_classes         = configs["NUM_CLASSES"],
        backbone_model_name = backbone)

    # ------------------
    # Validate & Test
    # ------------------
    val_metrics  = trainer_.validate(best_model, data_module, verbose=False)[0]
    test_metrics = trainer_.test(best_model, data_module, verbose=False)[0]

    # ------------------
    # Store results
    # ------------------
    results.append({
        "model": backbone,
        "checkpoint": best_ckpt_path,

        "val_loss":   val_metrics.get("val_loss"),
        "val_acc":    val_metrics.get("val_acc"),
        "val_f1":     val_metrics.get("val_f1"),
        "val_mAP":    val_metrics.get("val_mAP"),
        "val_auroc":  val_metrics.get("val_auroc"),

        "test_loss":  test_metrics.get("test_loss"),
        "test_acc":   test_metrics.get("test_acc"),
        "test_f1":    test_metrics.get("test_f1"),
        "test_map":   test_metrics.get("test_mAP"),
        "test_auroc": test_metrics.get("test_auroc")})



================ convnext_small =================

Loading pretrained [convnext_small] model
Downloading: "https://download.pytorch.org/models/convnext_small-0c510722.pth" to /root/.cache/torch/hub/checkpoints/convnext_small-0c510722.pth


100%|██████████| 192M/192M [00:01<00:00, 111MB/s] 
/usr/local/lib/python3.12/dist-packages/lightning_fabric/connector.py:571: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
/usr/local/lib/python3.12/dist-packages/torch/__init__.py:1551: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matm

Download complete!


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.12/dist-packages/pytorch_lightning/utilities/model_summary/model_summary.py:242: Precision 16-mixed is not supported by the model summary.  Estimated model size in MB will not be accurate. Using 32 bits instead.


┏━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name               ┃ Type                       ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ model              │ ConvNeXt                   │ 49.5 M │ train │     0 │
│ 1 │ loss_function      │ BCEWithLogitsLoss          │      0 │ train │     0 │
│ 2 │ accuracy_function  │ MultilabelAccuracy         │      0 │ train │     0 │
│ 3 │ f1_score_function  │ MultilabelF1Score          │      0 │ train │     0 │
│ 4 │ auroc_function     │ MultilabelAUROC            │      0 │ train │     0 │
│ 5 │ map_score_function │ MultilabelAveragePrecision │      0 │ train │     0 │
└───┴────────────────────┴────────────────────────────┴────────┴───────┴───────┘

...

Best checkpoint: /content/drive/MyDrive/ML Projects/Multi-label Diagnosis (ChestXRay)/Trained models/resnet50.ckpt
Loading pretrained [resnet50] model


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()